Goal: Compare time complexity of my df creation functions to find the most efficient method.

Findings:
- Don't use parseTime as a 'converter' in the `pd.read_csv` call. Takes 8 times as long otherwise.
- Use json_parser as converter for the columns we need to use json.loads() on. Saves ~10% of time.
- Implementing custom value_counts().to_dict() is much faster if we can guarantee the identity of the values (which we can, in our case)
- get_group is much faster on groups that are slices of their parent dataframe:
    - So this:
    ```python
        grouped = df[[field1, field2]]).groupby(field1)
        grouped.get_group(field1value)
    ```
    - can be considerably faster than this:
    ```python
        grouped = df.groupby(field1)
        grouped.get_group(field1value)
    ```


    

In [500]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from dateutil.parser import parse
from datetime import date

import sys
sys.path.append('../..')
from sf_lib.df import make_df_classify, make_df_tasks_with_props, make_df_vote_threshold, make_df_task0

%matplotlib inline

In [499]:
# reload sf import while I'm working on extracting functionality to it from notebooks
from importlib import reload
import sf_lib
reload(sf_lib)
reload(sf_lib.sf)
reload(sf_lib.df)


<module 'sf_lib.df' from '../..\\sf_lib\\df.py'>

In [436]:
object_info = pd.read_csv('../../catalogue/sf_spacefluff_object_data.csv', comment="#")
candidate_names_classify = np.loadtxt('../sf_candidate_names__classification-classify.txt', dtype='str')
catalogue_targets = np.loadtxt('../../catalogue/sf_catalogue_targets.txt', dtype='str')
answer_types = ['Galaxy', 'Group of objects (Cluster)', 'Something else/empty center']

In [502]:
%%timeit -n 1 -r 1
df = make_df_classify()

3.58 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [501]:
%%timeit -n 1 -r 1
make_df_tasks_with_props(df, candidate_names_classify, object_info)

3.34 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
